# **Customer propensity to purchase**


# **Introduction**

This is my first Kaggle, and my first foray into data analysis using python. The following kernel contains the steps, including approaches to this classification task, enumerated below for assessing the dataset:<br><br>


## Get the Dataset

To create a machine learning model, the first thing  we required is a dataset as machine learning model completly works on data. The collected data for a perticular problem in a proper format is known as the dataset.
Dataset may be different from one another to another according to purpose. Some common dataset formats are CSV,JSON,XLXS and more.

## Import libraries we required

depends on our requirements we need to use many libraries in python . The common libraries are pandas, numpy, Matplotlib, seaborn, scikit.pandas is a software library written for the Python programming language for data manipulation and analysis. numpy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays. and seaborn and matplot lib is usually used for data visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Fetch data

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/Users/praneethjakkala/Downloads/archive/training_sample.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
## analyse the data
data.head()

## Data Preprocessing

### Lets check our data how much noisy 

In [ ]:
data.isna().sum()

from above, we can identify there is no nan values in the data set

In [ ]:
data.info()

In [ ]:
X =data[['basket_icon_click', 'basket_add_list', 'basket_add_detail',
       'sort_by', 'image_picker', 'account_page_click', 'promo_banner_click',
       'detail_wishlist_add', 'list_size_dropdown', 'closed_minibasket_click',
       'checked_delivery_detail', 'checked_returns_detail', 'sign_in',
       'saw_checkout', 'saw_sizecharts', 'saw_delivery', 'saw_account_upgrade',
       'saw_homepage', 'device_mobile', 'device_computer', 'device_tablet',
       'returning_user', 'loc_uk']]
y=data['ordered']

## Feature selection

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(X.corr(),linewidth=2)
plt.show()

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X, 0.7)
len(set(corr_features))


In [ ]:
corr_features

In [ ]:
X = X.drop(corr_features,axis=1)
X

## Exploratory Data Analysis

How many users are purchased

In [ ]:
# 
ax = sns.countplot(data=data,x="ordered")
plt.show()

In [ ]:
dataframe=data

### Model creation

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [ ]:
print('Total no. of samples: Training and Testing dataset separately!')
print('X_train:', np.shape(X_train))
print('y_train:', np.shape(y_train))
print('X_test:', np.shape(X_test))
print('y_test:', np.shape(y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier7 = LogisticRegression()
classifier7.fit(X_train,y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
y_pred7 = classifier7.predict(X_test)
print(np.concatenate((y_pred7.reshape(len(y_pred7),1), y_test.values.reshape(len(y_test),1)),1))

### Metric Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
cm7 = confusion_matrix(y_test, y_pred7)
print(cm7)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=cm7, figsize=(6, 6), cmap=plt.cm.Greens)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
logreg=accuracy_score(y_test,y_pred7)
rocreg=roc_auc_score(y_test, y_pred7)
print('Accuracy : ',logreg,'\nROC Score : ',rocreg)

In [ ]:
import sklearn.metrics as metrics
print(metrics.classification_report(y_test, y_pred7))

### Hyper tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters_lr = [{'penalty':['l1','l2'],'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
grid_search_lr = GridSearchCV(estimator = classifier7,
                           param_grid = parameters_lr,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search_lr.fit(X_train, y_train)
best_accuracy_lr = grid_search_lr.best_score_
best_paramaeter_lr = grid_search_lr.best_params_  
print("Best Accuracy of LR: {:.2f} %".format(best_accuracy_lr.mean()*100))
print("Best Parameter of LR:", best_paramaeter_lr)

By using above parameter we get 99.26% accuracy

##  Testing

In [ ]:
classifierhyp = LogisticRegression(C=0.001,penalty='l2')
classifierhyp.fit(X_train,y_train)

In [ ]:
test_df= pd.read_csv("/Users/praneethjakkala/Downloads/archive/testing_sample.csv")

In [ ]:
test_df

In [ ]:
test=test_df[['basket_icon_click', 'basket_add_list', 'basket_add_detail', 'sort_by',
       'image_picker', 'account_page_click', 'promo_banner_click',
       'detail_wishlist_add', 'list_size_dropdown', 'closed_minibasket_click',
       'checked_delivery_detail', 'checked_returns_detail', 'sign_in',
       'saw_sizecharts', 'saw_delivery', 'saw_account_upgrade', 'saw_homepage',
       'device_mobile', 'device_tablet', 'returning_user', 'loc_uk']]

In [ ]:
test_df['result']=classifierhyp.predict(test)

In [ ]:
(test_df)

In [ ]:
test_df.result.value_counts()

In [ ]:
test_df.to_csv('results.csv')